In [4]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [5]:
reload(dl)

<module 'fitter.data_loader' from '../fitter/data_loader.pyc'>

In [6]:
def model_average(results):
    
    # calculate P( M_k | D )
    prob_Mk_given_D = lambda model_k : (
         np.exp(results[model_k]['logGBF']) / np.sum([np.exp(results[model_l]['logGBF']) for model_l in results.keys()]) 
    )
    
    # Get central value
    expct_y = 0
    for model in results.keys():
        expct_y += gv.mean(gv.gvar(results[model]['fit'])) *prob_Mk_given_D(model)
    
    # Get variance
    var_y = 0
    for model in results.keys():
        var_y += gv.var(gv.gvar(results[model]['fit'])) *prob_Mk_given_D(model)
    for model in results.keys():
        var_y += (gv.mean(gv.gvar(results[model]['fit'])))**2 *prob_Mk_given_D(model)
        
    var_y -= (expct_y)**2
    
    return gv.gvar(expct_y, np.sqrt(var_y))

In [7]:
colors = []

### Make fit comparison plot (saved to /tmp/)

In [8]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info(whose='mine')
other_results = data_loader.get_fit_info(whose='others')

In [9]:
# These axes compare fits
ax_fits = plt.axes([0.10,0.10,0.49,0.8])

y=0
labels = np.array([])
for results in [fit_results, other_results]:
    plt.axhline(y-0.5, ls='--')
    for name in sorted(results.keys()):
        x = gv.mean(gv.gvar(results[name]['fit']))
        xerr = gv.sdev(gv.gvar(results[name]['fit']))
        plt.errorbar(x=x, y=y, xerr=xerr, yerr=0.0,
                     fmt = 'ro', capsize=0.0, mec='white', ms=25.0, alpha=0.6,
                     ecolor='b', elinewidth=25.0)
        y = y + 1
        labels = np.append(labels, str(name))
        
        # Add band for FLAG
        if name == 'FLAG':
            plt.axvspan(x-xerr, x+xerr, alpha=0.3, color='g', label='FLAG')

ymax = y
            
# Show model average
avg = model_average(fit_results)
pm = lambda g, k : gv.mean(g) + k*gv.sdev(g)
plt.axvspan(pm(avg, -1), pm(avg, +1), alpha=0.3, color='m', label='model avg')
plt.axvline(pm(avg, 0), ls='-.', color='m')
        
plt.legend()

plt.yticks(range(len(labels)), labels, fontsize=15, rotation=65)
plt.ylim(-1, y)

#plt.title(particle+", "+abbr, fontsize=30)
plt.xlabel('$F_K/F_\pi$', fontsize=24)

plt.grid(ls='--')

# These axes compare the logGBFs
ax_logGBF = plt.axes([0.60,0.10,0.09,0.8])

# Get max logGBF
logGBF_max = np.nanmax([gv.mean(gv.gvar(fit_results[model]['logGBF'])) 
                       for model in fit_results.keys()])

y=0
labels = np.array([])
for results in [fit_results]:
    for name in sorted(results.keys()):
        logGBF = gv.mean(gv.gvar(results[name]['logGBF']))
        x = np.exp(logGBF - logGBF_max)
        
        plt.axvline(x, ls='--', alpha=0.4)
        plt.scatter(x=x, y=y, color='r')
        y = y + 1
        labels = np.append(labels, str(name))

plt.yticks([])
#plt.xlim(-0.05, 1.05)
plt.ylim(-1, ymax)
plt.xlabel("Norm\nGBF", fontsize=24)

# These axes compare the Q-values
ax_Q = plt.axes([0.70,0.10,0.09,0.8])

y=0
labels = np.array([])
for results in [fit_results]:
    for name in sorted(results.keys()):
        x = gv.mean(gv.gvar(results[name]['Q']))
        plt.axvline(x, ls='--', alpha=0.4)
        plt.scatter(x=x, y=y, color='r')
        y = y + 1
        labels = np.append(labels, str(name))

#for ti in np.arange(5)/4.0:
#    plt.axvline(ti, ls='--', alpha=0.2)

plt.yticks([])
plt.xlim(-0.05, 1.05)
plt.ylim(-1, ymax)
plt.xlabel("$Q$", fontsize=24)


# These axes compare the reduced chi2
ax_chi2 = plt.axes([0.80,0.10,0.1,0.8])

y=0
labels = np.array([])
for results in [fit_results]:
    for name in sorted(results.keys()):
        x = gv.mean(gv.gvar(results[name]['chi2/df']))
        plt.axvline(x, ls='--', alpha=0.4)
        plt.scatter(x=x, y=y, color='r')
        y = y + 1
        labels = np.append(labels, str(name))

#for ti in np.arange(9)/4.0:
#    plt.axvline(ti, ls='--', alpha=0.2)

plt.yticks([])
plt.xlim(0.25, 1.75)
plt.ylim(-1, ymax)
plt.xlabel(r"$\chi^2_\nu$", fontsize=24)

fig = plt.gcf()
plt.close()

data_loader.save_plots(fig, output_filename='fit_comparison')

Done.


### Make LEC comparison plot (saved to /tmp/)

In [10]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info(whose='mine')
other_results = data_loader.get_fit_info(whose='others')

In [11]:
lecs = ['L_4', 'L_5', # nlo terms
             'A_a', 'A_k', 'A_p', # nnlo terms
             'A_aa', 'A_ak', 'A_ap', # nnnlo terms
             'A_kk', 'A_kp', 'A_pp'] # more nnnlo terms

figs = []
for lec in lecs: 
    # These axes compare fits
    # These axes compare fits
    ax_fits = plt.axes([0.10,0.10,0.49,0.8])

    y=0
    labels = np.array([])
    for results in [fit_results]:
        plt.axhline(y-0.5, ls='--')
        for name in sorted(results.keys()):
            if lec in results[name].keys():
                x = gv.mean(gv.gvar(results[name][lec]))
                xerr = gv.sdev(gv.gvar(results[name][lec]))
                plt.errorbar(x=x, y=y, xerr=xerr, yerr=0.0,
                             fmt = 'ro', capsize=0.0, mec='white', ms=25.0, alpha=0.6,
                             ecolor='b', elinewidth=25.0)
                y = y + 1
                labels = np.append(labels, str(name))

    ymax = y


    plt.yticks(range(len(labels)), labels, fontsize=15, rotation=65)
    plt.ylim(-1, y)

    #plt.title(particle+", "+abbr, fontsize=30)
    plt.xlabel(lec, fontsize=24)

    plt.grid(ls='--')

    # These axes compare the logGBFs
    ax_logGBF = plt.axes([0.60,0.10,0.09,0.8])

    # Get max logGBF
    logGBF_max = np.nanmax([gv.mean(gv.gvar(fit_results[model]['logGBF'])) 
                           for model in fit_results.keys()])

    y=0
    labels = np.array([])
    for results in [fit_results]:
        for name in sorted(results.keys()):
            logGBF = gv.mean(gv.gvar(results[name]['logGBF']))
            x = np.exp(logGBF - logGBF_max)

            plt.axvline(x, ls='--', alpha=0.4)
            plt.scatter(x=x, y=y, color='r')
            y = y + 1
            labels = np.append(labels, str(name))

    plt.yticks([])
    #plt.xlim(-0.05, 1.05)
    plt.ylim(-1, ymax)
    plt.xlabel("Norm\nGBF", fontsize=24)
    
    # These axes compare the Q-values
    ax_Q = plt.axes([0.70,0.10,0.09,0.8])

    y=0
    labels = np.array([])
    for results in [fit_results]:
        for name in sorted(results.keys()):
            if lec in results[name].keys():
                x = gv.mean(gv.gvar(results[name]['Q']))
                plt.axvline(x, ls='--', alpha=0.4)
                plt.scatter(x=x, y=y, color='r')
                y = y + 1
                labels = np.append(labels, str(name))

    #for ti in np.arange(5)/4.0:
    #    plt.axvline(ti, ls='--', alpha=0.2)

    plt.yticks([])
    plt.xlim(-0.05, 1.05)
    plt.ylim(-1, ymax)
    plt.xlabel("$Q$", fontsize=24)


    # These axes compare the reduced chi2
    ax_chi2 = plt.axes([0.80,0.10,0.1,0.8])

    y=0
    labels = np.array([])
    for results in [fit_results]:
        for name in sorted(results.keys()):
            if lec in results[name].keys():
                x = gv.mean(gv.gvar(results[name]['chi2/df']))
                plt.axvline(x, ls='--', alpha=0.4)
                plt.scatter(x=x, y=y, color='r')
                y = y + 1
                labels = np.append(labels, str(name))

    #for ti in np.arange(9)/4.0:
    #    plt.axvline(ti, ls='--', alpha=0.2)

    plt.yticks([])
    plt.xlim(0.25, 1.75)
    plt.ylim(-1, ymax)
    plt.xlabel(r"$\chi^2_\nu$", fontsize=24)
    
    #plt.show()

    fig = plt.gcf()
    plt.close()
    figs.append(fig)

data_loader.save_plots(figs, output_filename='lec_comparison')

Done.
